<a href="https://colab.research.google.com/github/ahmeda335/FunctionCalling_OpenAIAPI/blob/main/PromptEngineeringGuide_FunctionCalling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is a starter kit for using ***Function Calling*** from OpenAI API.

### Installing Required Packages and Libraries 🌐

In [ ]:
!pip install termcolor --quiet
!pip install tenacity --quiet
!pip install openai --quiet

💫 Put your API key in colab Secrets Section then give it the the os environment with the name "OPENAI_API_KEY", as it is the default name for OpenAI API.

💫 If You want to use another name you can do put change the default then..<br>
---->  client = OpenAI( api_key=os.environ.get("CUSTOM_ENV_NAME"), )

💫 Initialize the model you want to use.

In [ ]:
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import json
from google.colab import userdata
import os

key = userdata.get('OPENAI-API-KEY')
os.environ['OPENAI_API_KEY'] = key

client = OpenAI()
model = "gpt-4o-mini"  # TODO: Searching for free models to use. "Nothing till now 🥲"

Initializing the functions to be chosen using the model 🎯

In [ ]:
def downloading_video(video_url, quality):
  print("Downloading video from", video_url, "with quality", quality)
  return


def downloading_audio(audio_url, quality):
  print("Downloading audio from", audio_url, "with quality", quality, "only audio")
  return


def downloading_playlist_video(playlist_url, qulaity):
  print("Downloading playlist from", playlist_url, "with quality", qulaity)
  return


def downloading_playlist_audio(playlist_url, qulaity):
  print("Downloading playlist from", playlist_url, "with quality", qulaity, "only audio")
  return


### The main function to connect the model to the user 🎗️

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=30), stop=stop_after_attempt(3)) # This line to retry the function each random time between 1 and 40 seconds with maximum 3 attempts. To increase the Robustness of the code.
def chat_completion_request(messages, tools=None, tool_choice=None, model=model):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

💫 ( tools ) is used to descripe the functions and its parameters to the model such that it can choose from them depending on the user demand.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "downloading_video",
            "description": "This function is used to download a single video from youtube with a chosen quality from the user and the url of the video",
            "parameters": {
                "type": "object",
                "properties": {
                    "video_url": {
                        "type": "string",
                        "description": "The url of the video",
                    },
                    "quality": {
                        "type": "string",
                        "description": "The quality of the vidio, e.g. '480p'",
                    },
                },
                "required": ["video_url", "quality"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "downloading_audio",
            "description": "This function is used to download a single audio from youtube with a chosen quality from the user and the url of the audio",
            "parameters": {
                "type": "object",
                "properties": {
                    "audio_url": {
                        "type": "string",
                        "description": "The url of the audio",
                    },
                    "quality": {
                        "type": "string",
                        "description": "The quality of the audio",
                    },
                },
                "required": ["audio_url", "quality"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "downloading_playlist_video",
            "description": "This function is used to download a playlist of videos from youtube with a chosen quality from the user and the url of the playlist",
            "parameters": {
                "type": "object",
                "properties": {
                    "playlist_url": {
                        "type": "string",
                        "description": "The url of the playlist",
                    },
                    "quality": {
                        "type": "string",
                        "description": "The quality of the playlist videos, e.g. '480p'",
                    },
                },
                "required": ["playlist_url", "quality"],
                "additionalProperties": False,
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "downloading_playlist_audio",
            "description": "This function is used to download a playlist of audios from youtube with a chosen quality from the user and the url of the playlist",
            "parameters": {
                "type": "object",
                "properties": {
                    "playlist_url": {
                        "type": "string",
                        "description": "The url of the playlist",
                    },
                    "quality": {
                        "type": "string",
                        "description": "The quality of the playlist audios",
                    },
                },
                "required": ["playlist_url", "quality"],
                "additionalProperties": False,
            },
        }
    },
]

 💫 This lovely pretty_print_conversation function from ***Colin Jarvis*** in the OpenAI CookBook changes the color of the text depending on who is speaking: the user, the system, or the assistant. And also depending on the response of the assistant 🫡





In [ ]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))

💫 Cases to handle from the response of the model.

In [ ]:
def handle_length_error(response):
  print("Handling length error.")
  return


def handle_content_filter_error(response):
  print("Handling content filter error.")
  return


# def handle_tool_call(response):
#   print("Handling tool call.")
#   return


# def handle_normal_response(response):
#   print("Handling normal response.")
#   return


def handle_unexpected_case(response):
  print("Handling unexpected case.")
  return


def handling_edge_cases():
  # Check if the conversation was too long for the context window
  if response['choices'][0]['message']['finish_reason'] == "length":
      print("Error: The conversation was too long for the context window.")
      # Handle the error as needed, e.g., by truncating the conversation or asking for clarification
      handle_length_error(response)


  # Check if the model's output included copyright material (or similar)
  if response['choices'][0]['message']['finish_reason'] == "content_filter":
      print("Error: The content was filtered due to policy violations.")
      # Handle the error as needed, e.g., by modifying the request or notifying the user
      handle_content_filter_error(response)

  # # Check if the model has made a tool_call. This is the case either if the "finish_reason" is "tool_calls" or if the "finish_reason" is "stop" and our API request had forced a function call
  # if (response['choices'][0]['message']['finish_reason'] == "tool_calls" or
  #     # This handles the edge case where if we forced the model to call one of our functions, the finish_reason will actually be "stop" instead of "tool_calls"
  #     (our_api_request_forced_a_tool_call and response['choices'][0]['message']['finish_reason'] == "stop")):
  #     # Handle tool call
  #     print("Model made a tool call.")
  #     # Your code to handle tool calls
  #     handle_tool_call(response)

  # # Else finish_reason is "stop", in which case the model was just responding directly to the user
  # elif response['choices'][0]['message']['finish_reason'] == "stop":
  #     # Handle the normal stop case
  #     print("Model responded directly to the user.")
  #     # Your code to handle normal responses
  #     handle_normal_response(response)



In [ ]:
def main():
  messages = []
  messages.append({"role": "system", "content": "You are a helpful assistant, determine what the user wants to download from the youtube: video, audio, playlist of videos, or playlist of audios"})


  while True:
      messages.append({"role": "user", "content": input(" ")})
      response = chat_completion_request(
        messages=messages,
        tools=tools,
    )

      handling_edge_cases()

      if  response.choices[0]["finish_reason"] == "function_call":
          fn_name = response.choices[0].message["function_call"].name
          arguments = jsno.load(response.choices[0].message["function_call"].arguments)

          function = locals()[fn_name]
          result = function(**arguments)

          messages.append(
              {
                  "role": "assistant",
                  "content": None,
                  "function_call": {
                      "name": fn_name,
                      "arguments": arguments,
                  },
              }
          )

          messages.append(
              {
                  "role": "function",
                  "name": fn_name,
                  "content": f'{{"result": {str(result)} }}'}
          )
          response = chat_completion_request(
           messages=messages,
           tools=tools,
        )
          handling_edge_cases()
          pretty_print_conversation(messages)
          break

      elif response.tool_calls is None:
          messages.append({"role": "assistant", "content": response.choices[0].message.content})
          pretty_print_conversation(messages)

      else:   # Any other unexpected cases.
        print("Unexpected finish_reason:", response['choices'][0]['message']['finish_reason'])
        # Handle unexpected cases as needed
        handle_unexpected_case(response)




In [ ]:
main()